In [2]:
from google.colab import drive
from google.colab.patches import *
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install easyocr
!pip install ultralytics
!pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.2/645.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=2513017024dc6d672630118158d3ca3227657be13fdb4061df3fadc0f8a10261
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy


In [6]:
!cp /content/drive/MyDrive/modules/util.py /content

In [7]:
# !cp /content/drive/MyDrive/modules/util2.py /content

In [8]:
!cp /content/drive/MyDrive/modules/sort.py /content

In [9]:
!cp /content/drive/MyDrive/modules/visualize.py /content

In [10]:
!cp /content/drive/MyDrive/modules/add_missing_data.py /content

In [11]:
from ultralytics import YOLO
import cv2, os, glob
import numpy as np, pandas as pd
import matplotlib.pyplot as plt



from sort import *
from util import *
# from util2 import *

import easyocr

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [12]:
cocomodel=YOLO('yolov8n.pt')
# print(cocomodel)

100%|██████████| 6.23M/6.23M [00:00<00:00, 78.0MB/s]


In [13]:
license_plate_detector=YOLO("/content/drive/MyDrive/model_weights/license_plate_detector.pt")
# print(license_plate_detector)

In [14]:
vehicle=[2,3,4,6,8]         #list of label id for vechil detection

mater_tracker=Sort()        #used for tracking vechiles

resultes={}

# crop_lp=[]
# crop_lp_list_gray=[]
# crop_lp_list_threshold=[]

In [ ]:
# video test
vid=cv2.VideoCapture("/content/drive/MyDrive/google_colab_datasets/input_videos/ANPR (number plate)/demo.mp4")
ret,frame_nmr=True,-1
while(ret):
    frame_nmr+=1
    ret,frame=vid.read()
    print(ret,frame_nmr)
    if(ret):
        # if(frame_no>10): break
        resultes[frame_nmr]={}

        # detect vechile
        detections= cocomodel(frame)[0]
        detections_list=[]
        # print(detections)
        # detectic  vechile
        for detection in detections.boxes.data.tolist():
            x1,y1,x2,y2,score,class_id=detection
            if(int(class_id) in vehicle):
                detections_list.append([x1,y1,x2,y2,score])

        print(detections_list)       #degugging
        if(len(detections_list)==0): continue
        # track vechiles
        track_id=mater_tracker.update(np.asarray(detections_list))
        print(track_id)
        # detect license plate
        license_plates=license_plate_detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            # print(license_plate)
            x1,y1,x2,y2,score,class_id=license_plate
            # print(type(x1),type(y1),type(x2),type(y2),type(score),type(class_id)

            # assign licences plate to car
            xcar1,ycar1,xcar2,ycar2,car_id=get_car(license_plate,track_id)
            if car_id!=-1:
              # corp license plate
              corp_license_plate=frame[int(y1):int(y2),int(x1):int(x2),:]
              # process licens plate
              corp_license_plate_gray=cv2.cvtColor(corp_license_plate,cv2.COLOR_BGR2GRAY)
              _,corp_license_plate_therhold=cv2.threshold(corp_license_plate,64,255,cv2.THRESH_BINARY_INV)

              # cv2.imshow("orignal_crop",corp_license_plate)
              # cv2.imshow("thresh_orignal_crop",corp_license_plate_therhold)

              # crop_lp.append(corp_license_plate)
              # crop_lp_list_gray.append(corp_license_plate_gray)
              # crop_lp_list_threshold.append(corp_license_plate_therhold)

              # print(corp_license_plate_therhold)            #debuging

              # read lincese plate
              license_plate_text,license_plate_score=read_license_plate(corp_license_plate_therhold)
              # print(license_plate_text,license_plate_score)     #debugging


              if(license_plate_text is not None):
                  resultes[frame_nmr][car_id]={'car':{'bbox':[xcar1,ycar1,xcar2,ycar2]},
                                            'license_plate':{'bbox':[ x1,y1,x2,y2],'text':license_plate_text,'bbox_score': score,'text_score':license_plate_score}}

# write resulte
write_csv(resultes,"/content/drive/MyDrive/google_colab_datasets/output/csv/test2.csv")

vid.release()


0: 416x640 2 cars, 121.4ms
Speed: 2.2ms preprocess, 121.4ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 640)



True 0
[[80.4343490600586, 98.14193725585938, 257.4587707519531, 228.9402313232422, 0.9036537408828735], [107.48316955566406, 227.2660675048828, 949.0166015625, 664.0021362304688, 0.8898075222969055]]
[]


0: 416x640 3 License_Plates, 114.3ms
Speed: 3.7ms preprocess, 114.3ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 2 cars, 95.6ms
Speed: 2.1ms preprocess, 95.6ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 640)



True 1
[[78.72087097167969, 96.77953338623047, 262.2138977050781, 232.6151885986328, 0.9115791320800781], [112.53455352783203, 227.21328735351562, 945.6424560546875, 657.03466796875, 0.8873939514160156]]
[]


0: 416x640 2 License_Plates, 117.4ms
Speed: 2.1ms preprocess, 117.4ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 2 cars, 98.4ms
Speed: 2.0ms preprocess, 98.4ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 640)



...

True 629
[[44.90151596069336, 123.02916717529297, 1060.595703125, 621.8558349609375, 0.9345344305038452], [115.4692611694336, 19.94969940185547, 253.40528869628906, 132.91635131835938, 0.7187339663505554], [370.3183898925781, 14.829547882080078, 448.53497314453125, 82.91661834716797, 0.633881151676178]]
[[     371.71      16.307      452.96      84.617         381]
 [     108.34      19.331      262.52      136.14         380]
 [     27.439      109.68      1086.7      638.69         313]]


0: 416x640 1 License_Plate, 124.8ms
Speed: 2.1ms preprocess, 124.8ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 3 cars, 1 truck, 103.6ms
Speed: 3.1ms preprocess, 103.6ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 640)



True 630
[[51.393165588378906, 123.44849395751953, 1054.2913818359375, 616.539794921875, 0.9447980523109436], [390.922119140625, 13.633857727050781, 469.0372009277344, 82.89582824707031, 0.7739856839179993], [125.71495056152344, 19.646770477294922, 268.6513977050781, 135.31944274902344, 0.7641693353652954]]
[[     390.46      15.415      472.04      84.988         381]
 [     122.41      19.947      275.63      138.07         380]
 [     31.646      111.61      1076.6      631.25         313]]


0: 416x640 2 License_Plates, 138.9ms
Speed: 2.6ms preprocess, 138.9ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 640)


False 631
{'car': {'bbox': [1.9754228476449498, 224.34059001189306, 735.9101288971455, 685.6938807049589]}, 'license_plate': {'bbox': [267.9521789550781, 526.6824951171875, 493.9193420410156, 619.4752807617188], 'text': 'T656XA', 'bbox_score': 0.7282096147537231, 'text_score': 0.44162578582347667}}
{'car': {'bbox': [10.047782801678977, 251.81334881357324, 707.257610451892, 685.0134404290882]}, 'license_plate': {'bbox': [271.7147521972656, 537.8772583007812, 486.6166076660156, 617.5762939453125], 'text': 'D656YH', 'bbox_score': 0.6827227473258972, 'text_score': 0.15539811574163134}}
{'car': {'bbox': [58.813842198339614, 333.0705522343718, 701.6268467857342, 709.4280826532769]}, 'license_plate': {'bbox': [269.3071594238281, 598.6428833007812, 462.68621826171875, 672.192138671875], 'text': 'L656XH', 'bbox_score': 0.6859304904937744, 'text_score': 0.618759707981908}}
{'car': {'bbox': [57.8871255662919, 346.7914424674652, 694.095249186387, 714.9459335562015]}, 'license_plate': {'bbox': [265

In [15]:
df=pd.read_csv("/content/drive/MyDrive/google_colab_datasets/output/csv/test.csv")
df.head(10)

,frame_nmr,car_id,car_bbox,license_plate_bbox,license_plate_bbox_score,license_number,license_number_score
0,0,2820,[752.8721923828125 1369.210693359375 1430.7805...,[960.5645751953125 1767.7939453125 1176.414306...,0.422610,2NAI3NRU,0.261607
1,0,2822,[2197.166015625 1154.0797119140625 2785.649414...,[2420.345947265625 1539.788818359375 2571.8488...,0.362390,NLEISU,0.244411
2,1,2820,[752.7639685690319 1369.2585681977473 1430.436...,[960.5538940429688 1767.7606201171875 1176.891...,0.423722,NA13NPU,0.174168
3,1,2822,[2197.3992701546013 1154.2835435957104 2785.55...,[2420.40625 1539.7877197265625 2572.0717773437...,0.363599,NLEISU,0.350112
4,2,2820,[750.1345761318507 1374.3687354403723 1425.920...,[950.4424438476562 1755.72509765625 1206.49584...,0.273883,2NAI3NRU,0.147421
5,2,2822,[2201.1788267215284 1163.4534636892133 2783.90...,[2422.310546875 1540.1971435546875 2572.301269...,0.363493,HUIWSU,0.058274
6,3,2820,[748.0783757034214 1378.2311500660096 1424.694...,[951.6421508789062 1770.8243408203125 1153.712...,0.500443,NAI?NRL,0.321454
7,3,2822,[2196.5316135798007 1169.006615954218 2783.134...,[2422.029052734375 1544.68505859375 2578.20166...,0.369616,N51YSU,0.368196
8,4,2820,[749.085140201071 1380.2563205588128 1430.1694...,[947.2576904296875 1752.593994140625 1210.2465...,0.272309,NAI?NPU,0.405608
9,4,2822,[2196.699919156056 1171.5291397548656 2784.998...,[2422.6142578125 1557.7474365234375 2589.08569...,0.366089,HU51KSU,0.427908


In [ ]:
!python add_missing_data.py

['122', '125', '131', '132', '135', '138', '140', '141', '143', '149', '151', '153', '155', '158', '168', '169', '172', '173', '174', '175', '176', '178', '179', '180', '181', '182', '186', '187', '188', '190', '191'] 228
['264', '267', '271', '276'] 261
['322'] 300
['424', '491', '506', '509', '511', '514', '516', '517', '519', '532', '534', '547', '567', '568', '569', '571', '572', '573'] 313


In [ ]:
!python visualize.py